In [ ]:
# Sentiment Analysis Model Development

In [ ]:
## Loading

In [ ]:
df = pd.read_csv('eda_data.csv')

In [ ]:
# Function to create and return the Bag of Words DataFrame

In [ ]:
def create_bow_representation(texts, use_tfidf=False, max_features=100):

In [ ]:
    vectorizer = TfidfVectorizer(max_features=max_features) if use_tfidf else CountVectorizer(max_features=max_features)

In [ ]:
    X = vectorizer.fit_transform(texts)

In [ ]:
    feature_names = vectorizer.get_feature_names_out()

In [ ]:
    df_bow = pd.DataFrame(X.toarray(), columns=feature_names)

In [ ]:
    return df_bow, feature_names

In [ ]:
df_bow, feature_names = create_bow_representation(df['cleaned_text'], use_tfidf=False, max_features=50)

In [ ]:
def visualize_bow(df_bow, feature_names, annotate=False):

In [ ]:
    plt.figure(figsize=(12, 8))

In [ ]:
    sns.heatmap(df_bow, annot=annotate, cmap='viridis', cbar=False, xticklabels=feature_names)

In [ ]:
    plt.title('Bag of Words Matrix')

In [ ]:
    plt.xlabel('Features')

In [ ]:
    plt.ylabel('Documents')

In [ ]:
    plt.show()

In [ ]:
visualize_bow(df_bow, feature_names, annotate=False)

In [ ]:
# Logistic Regression Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
y = df['class']

In [ ]:
encoder = LabelEncoder()